In [1]:
import matplotlib.pyplot as plt
import numpy as np
from KDEpy import NaiveKDE, FFTKDE
import scipy.interpolate as interp
import scipy.integrate as integrate
from function_library_3 import extract
from tqdm import tqdm

#================================================================================
# SETUP

#change default font size for graphs
plt.rcParams.update({'font.size':20})

# create the needed directories and clear old output
import os,glob
try:
    os.mkdir('1_KDE_Data')
except:
    for filename in glob.glob('1_KDE_Data/*'):
        os.remove(filename)
        
try:
    os.mkdir('2_Stacked_Heights')
except:
    for filename in glob.glob('2_Stacked_Heights/*'):
        os.remove(filename)
try:
    os.mkdir('2_Basin_Pressures')
except:
    for filename in glob.glob('2_Basin_Pressures/*'):
        os.remove(filename)
try:
    os.mkdir('2_Basin_Temperatures')
except:
    for filename in glob.glob('2_Basin_Temperatures/*'):
        os.remove(filename)

#================================================================================
# CREATE FUNCTION THAT EXTRACTS DATA FROM EACH BASIN

def data_extract(basin):
    # open tracer file
    file0    = open('1_Basin_Data/{}.dat'.format(basin),'r')
    time_0, time_f, save_step, R_p, M_p, R_imp, v_imp, coords, R_c = eval(file0.readline().replace('\n',''))
    used = extract(file0)
    material = extract(file0)
    volume   = extract(file0)
    spread   = extract(file0)
    mass     = extract(file0)
    launch_time     = extract(file0)
    launch_polar    = extract(file0)
    launch_length   = extract(file0)
    launch_height   = extract(file0)
    launch_velocity = extract(file0)
    launch_angle    = extract(file0)
    pressure    = extract(file0)
    temperature = extract(file0)
    landing_time   = extract(file0)
    landing_polar  = extract(file0)
    landing_length = extract(file0)
    max_altitude   = extract(file0)
    file0.close()    
    
    N,E = coords
    
    weight_choice = volume
    
    # Accumulated Height (Weight must be volume for this to work)
    height_x, height_prime = NaiveKDE(kernel='gaussian', bw='silverman').fit(landing_length,weights=volume).evaluate()
    height_x = list(height_x)
    height_val = [1e-16]
    total_volume = sum(volume)
    for i in range(1,len(height_x)):
        height_val.append(height_prime[i]*total_volume/(height_x[i]-height_x[i-1])/(2*np.pi*R_p*np.sin(height_x[i]/R_p)))
    
    # Pressures
    pressure_x, pressure_val = NaiveKDE(kernel='gaussian', bw='silverman').fit(pressure,weights=weight_choice).evaluate()
    # Temperature
    temperature_x, temperature_val = NaiveKDE(kernel='gaussian', bw='silverman').fit(temperature,weights=weight_choice).evaluate()
    
    
    #2D KDE
    data0 = np.array([np.log10(abs(landing_length)),np.log10(pressure)])
    land_pres_log=np.transpose(data0) # log data of pressure and landing location
    data0 = np.array([np.log10(abs(landing_length)),np.log10(temperature)])
    land_temp_log=np.transpose(data0) # log data of temperature and landing location
    
    grid_points = 2**6
    # Compute the kernel density estimate
    kde = FFTKDE(kernel='gaussian', bw=.1)
    grid_p, points_p = kde.fit(land_pres_log, weights=weight_choice).evaluate(grid_points)
    grid_T, points_T = kde.fit(land_temp_log, weights=weight_choice).evaluate(grid_points)

    # Generate the 2D grid of points and their Z values
    Xp_log, Yp_log = np.unique(grid_p[:, 0]), np.unique(grid_p[:, 1])
    X = 10**(Xp_log) ; Yp = 10**(Yp_log)
    Zp = points_p.reshape(grid_points, grid_points).T
    XT_log, YT_log = np.unique(grid_T[:, 0]), np.unique(grid_T[:, 1])
    pass             ; YT = 10**(YT_log)
    ZT = points_T.reshape(grid_points, grid_points).T   
    
    # change the mesh into three lists of xyz coordinates
    x = [] # get x coordinates
    number = len(X)
    for i in range(number):
        for each in X:
            x.append(each) 
    # get y and z coords
    yp,zp=[],[] ;yT,zT=[],[]   
    for each in Yp:
        for i in range(number):
            yp.append(each)   
    for each in Zp:
        for every in each:
            zp.append(every)   
    for each in YT:
        for i in range(number):
            yT.append(each)   
    for each in ZT:
        for every in each:
            zT.append(every)
    
    #-----------------------------------------------
    #            Graph the Basin Data
    #-----------------------------------------------
    
    # Stacked Height
    fig = plt.figure(figsize=[16,8],facecolor='white')
    plt.suptitle("{}".format(basin), fontsize=32)
    plt.title('Stacked Height of Ejecta from Basin Center to Antipode')
    plt.plot(height_x,height_val,label='Stacked Height')
    plt.xlabel('Landing Location')
    plt.xlim(0,np.pi*R_p)
    plt.xticks([0,R_p*np.pi/4,R_p*np.pi/2,3*R_p*np.pi/4, R_p*np.pi],labels=['Basin Center','45°','90°','135°','Antipode'])
    plt.ylabel('Height [km]')
    plt.ylim(bottom=5e-4)
    plt.yscale('log') 
    plt.grid(True, ls='--', zorder=-15); plt.legend();
    fig.savefig('2_Stacked_Heights/Stacked Height of {}'.format(basin))
    plt.close(fig)   
    
    #-----------------------------------------------
    
    # Pressure Histogram
    fig = plt.figure(figsize=[16,8],facecolor='white')
    plt.suptitle("{}".format(basin), fontsize=32)
    plt.title('Peak Pressure Distribition')
    plt.hist(pressure, bins = 500, alpha=.2, weights=weight_choice, density=True)
    plt.plot(pressure_x,pressure_val, label='KDE')
    plt.xlabel('Pressure [GPa]')
    plt.ylabel('Probability Density')
    plt.grid(True, ls='--', zorder=-15); plt.legend(); 
    fig.savefig('2_Basin_Pressures/Pressure Distribution of {}'.format(basin))
    plt.close(fig)
    
    # Temperature Histogram
    fig = plt.figure(figsize=[16,8],facecolor='white')
    plt.suptitle("{}".format(basin), fontsize=32)
    plt.title('Peak Temperature Distribition')
    plt.hist(temperature, bins = 500, alpha=.2, weights=weight_choice, density=True)
    plt.plot(temperature_x,temperature_val, label='KDE')
    plt.xlabel('Temperature [K]')
    plt.ylabel('Probability Density')
    plt.grid(True, ls='--', zorder=-15); plt.legend(); 
    fig.savefig('2_Basin_Temperatures/Temperature Distribution of {}'.format(basin))
    plt.close(fig)
    
    #-----------------------------------------------
    # (  These graphs can be very slow if grid_points are set sufficiently high (> 2**7)  )
    
    # Pressure 2D KDE
    fig = plt.figure(figsize=[16,8],facecolor='white')
    plt.suptitle("{}".format(basin), fontsize=32)
    plt.title('Peak Pressure vs Landing Location PDF')
    plt.pcolor(X, Yp, Zp, cmap="viridis")
    plt.xlabel('Landing Location [km]') 
    plt.xlim(min(landing_length),np.pi*R_p)
    plt.xscale('log') 
    plt.xticks([100,300,1000,3000],labels=[100,300,1000,3000])   
    plt.ylabel('Pressure [GPa]')
    plt.ylim(min(pressure),max(pressure))
    plt.yscale('log')
    plt.colorbar(label='Abundance [km GPa]')
    fig.savefig('2_Basin_Pressures/Pressure 2D KDE of {}'.format(basin))
    plt.close(fig)
    
    # Temperature 2D KDE
    fig = plt.figure(figsize=[16,8],facecolor='white')
    plt.suptitle("{}".format(basin), fontsize=32)
    plt.title('Peak Temperature vs Landing Location PDF')
    plt.pcolor(X, YT, ZT, cmap="viridis")
    plt.xlabel('Landing Location [km]') 
    plt.xlim(min(landing_length),np.pi*R_p)
    plt.xscale('log') 
    plt.xticks([100,300,1000,3000],labels=[100,300,1000,3000])   
    plt.ylabel('Temperature [K]')
    plt.ylim(min(temperature),max(temperature))
    plt.yscale('log')
    plt.colorbar(label='Abundance [km K]')
    fig.savefig('2_Basin_Temperatures/Temperature 2D KDE of {}'.format(basin))
    plt.close(fig)
    
    return [basin, x, yp,zp, yT,zT, height_x, height_val, coords, R_p, R_c]
    
#================================================================================
# CREATE MAIN FUNCTION 

def data_prime():

    basin_list = [] # generate a list of the included basins
    for filename in os.listdir('1_Basin_Data'):
        if filename.endswith(".dat"):
            basin_list.append(filename[:-4])
            
    basin_list.sort() # alphabatize the basins
        
    # make output files for each basin
    for basin in tqdm(basin_list, desc = 'Generating KDEs'):
        #print(basin)
        KDEfile = open('1_KDE_Data/{}.dat'.format(basin),'a')
        KDEfile.write(str(list(data_extract(basin))))  
        KDEfile.close()

#================================================================================
# RUN MAIN FUNCTION 

data_prime()

print('COMPLETED')


Generating KDEs:   0%|                                   | 0/19 [00:00<?, ?it/s]/Users/sorenhelhoski/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:176: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
/Users/sorenhelhoski/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:192: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
Generating KDEs:   5%|█▍                         | 1/19 [00:01<00:29,  1.66s/it]/Users/sorenhelhoski/anaconda3/lib/python3

/Users/sorenhelhoski/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:192: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
Generating KDEs:  53%|█████████████▋            | 10/19 [00:15<00:14,  1.59s/it]/Users/sorenhelhoski/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:176: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
/Users/sorenhelhoski/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:192: MatplotlibDeprecationWarning: shadin

Generating KDEs: 100%|██████████████████████████| 19/19 [00:30<00:00,  1.59s/it]

COMPLETED
